In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [ ]:
nyc_cd = gpd.read_file('../data/Community_Districts.geojson')
print(nyc_cd.crs)
nyc_cd.head()

In [ ]:
type(nyc_cd)

In [ ]:
nyc_cd.loc[0, 'geometry']

In [ ]:
print(nyc_cd.loc[0, 'geometry'])

In [ ]:
nyc_cd.plot();

In [ ]:
leg_kwds = {'title': 'New York City Community Districts', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 2}

nyc_cd.plot(column = 'boro_cd', figsize=(10, 10),
              edgecolor = 'black',
              legend = True, legend_kwds = leg_kwds, 
              cmap = 'Set2')
plt.show()

First let's add the Farmers Market data

In [ ]:
farmers_market = pd.read_csv('../data/DOHMH_Farmers_Markets.csv')
print(farmers_market.shape)
farmers_market.head(3)

In [ ]:
farmers_market = farmers_market[['Borough','Market_Name','Street_Address','Community_District','Latitude','Longitude','Location_Point']]

In [ ]:
farmers_market.head()

In [ ]:
farmers_market.columns = ['Borough','Market_Name','Street_Address','Community_District_FM','Latitude','Longitude','Location_Point']

In [ ]:
farmers_market.head()

In [ ]:
farmers_market.info()

In [ ]:
farmers_market['geometry'] = farmers_market.apply(lambda x: Point((x.Longitude, 
                                                         x.Latitude)), 
                                        axis=1)
farmers_market.head(3)

In [ ]:
fm_geo = gpd.GeoDataFrame(farmers_market, 
                           crs = nyc_cd.crs, 
                           geometry = farmers_market['geometry'])

In [ ]:
type(fm_geo)

In [ ]:
nyc_cd.crs

Now let's add the healthy stores data

In [ ]:
healthy_stores = pd.read_csv('../data/Recognized_Shop_Healthy_Stores.csv')
print(healthy_stores.shape)
healthy_stores.head(3)

In [ ]:
healthy_stores = healthy_stores[['Store_Name','Street_Address','Borough','Latitude','Longitude','Community_District']]

In [ ]:
healthy_stores.head()

In [ ]:
healthy_stores.info()

In [ ]:
healthy_stores['geometry'] = healthy_stores.apply(lambda x: Point((x.Longitude, 
                                                         x.Latitude)), 
                                        axis=1)
healthy_stores.head(3)

In [ ]:
hs_geo = gpd.GeoDataFrame(healthy_stores, 
                           crs = nyc_cd.crs, 
                           geometry = healthy_stores['geometry'])

In [ ]:
type(hs_geo)

Finally let's add the SNAP Centers

In [ ]:
snap = pd.read_csv('../data/Directory_of_SNAP_Centers.csv')
print(snap.shape)
snap.head(3)

In [ ]:
snap['geometry'] = snap.apply(lambda x: Point((x.Longitude, 
                                                         x.Latitude)), 
                                        axis=1)
snap.head(3)

In [ ]:
nyc_cd.crs

In [ ]:
snap_geo = gpd.GeoDataFrame(snap, 
                           crs = nyc_cd.crs, 
                           geometry = snap['geometry'])

Do we want to rename the Community District Column? Let's think about this before we do a merge or geospatial join.

Let's also drop the unecessary columns now that we fixed the issuse from before.

In [ ]:
type(snap_geo)

In [ ]:
snap.head()

In [ ]:
snap = snap[['Borough','Facility_Name','Street_Address','Latitude','Longitude','Community_District','geometry']]

In [ ]:
snap.head()

In [ ]:
snap.columns = ['Borough','Facility_Name','Street_Address','Latitude','Longitude','Community_District_SP','geometry']

In [ ]:
snap.head()

In [ ]:
nyc_cd = nyc_cd[['boro_cd', 'geometry']]

In [ ]:
nyc_area_center = ['40.71','-74.00']
print(nyc_area_center)  

In [ ]:
map_nyc = folium.Map(location = nyc_area_center, zoom_start = 12)
map_nyc

We will need iterrows for the farmers markets, healthy stores, and snapcenters

In [ ]:
i = 0
for row_index, row_values in farmers_market.iterrows():
    if i <3: 
       print('index is', row_index)
       print('values are:')
       print(' ')
       print(row_values)
       print('------------------------- ')
       i+=1

In [ ]:
i = 0
for row_index, row_values in healthy_stores.iterrows():
    if i <3: 
       print('index is', row_index)
       print('values are:')
       print(' ')
       print(row_values)
       print('------------------------- ')
       i+=1

In [ ]:
i = 0
for row_index, row_values in snap.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

In [ ]:
map_nyc = folium.Map(location =  nyc_area_center, zoom_start = 12)


for row_index, row_values in farmers_market.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Market_Name'])
    icon=folium.Icon(color="green",icon="leaf", prefix='fa')


for row_index, row_values in healthy_stores.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Store_Name'])
    icon=folium.Icon(color="orange",icon="shopping-cart", prefix='fa')

for row_index, row_values in snap.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Facility_Name'])
    icon=folium.Icon(color="red",icon="institution", prefix='fa')

marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

marker.add_to(map_nyc)
map_nyc.save('../maps/healthyfoodaccessnyc.html')


map_nyc

In [ ]:
cluster_map_nyc = folium.Map(location =  nyc_area_center, zoom_start = 12)

marker_cluster = MarkerCluster().add_to(cluster_map_nyc)


for row_index, row_values in farmers_market.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Market_Name'])
    icon=folium.Icon(color="green",icon="leaf", prefix='fa')


for row_index, row_values in healthy_stores.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Store_Name'])
    icon=folium.Icon(color="orange",icon="shopping-cart", prefix='fa')

for row_index, row_values in snap.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Facility_Name'])
    icon=folium.Icon(color="red",icon="institution", prefix='fa')

marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

marker.add_to(marker_cluster)

cluster_map_nyc.save('../maps/healthyfoodaccessnyccluster.html')

cluster_map_nyc

Lets make a map of the community district witht the most facilities.